<a href="https://colab.research.google.com/github/liuxiu842/ML/blob/master/RepeatBuyers_xgboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

import gc
import pandas as pd
# 用户行为，使用format1进行加载
# 加载全量样本
"""
user_log = pd.read_csv('./data_format1/user_log_format1.csv', dtype={'time_stamp':'str'})
user_info = pd.read_csv('./data_format1/user_info_format1.csv')
train_data1 = pd.read_csv('./data_format1/train_format1.csv')
submission = pd.read_csv('./data_format1/test_format1.csv')
"""
# 加载小样本
user_log = pd.read_csv('/content/drive/My Drive/Colab Notebooks/L11/data_format1_small/sample_user_log.csv', dtype={'time_stamp':'str'})
user_info = pd.read_csv('/content/drive/My Drive/Colab Notebooks/L11/data_format1_small/sample_user_info.csv')
train_data1 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/L11/data_format1_small/train.csv')
submission = pd.read_csv('/content/drive/My Drive/Colab Notebooks/L11/data_format1_small/test.csv')
#train_data = pd.read_csv('./data_format2/train_format2.csv')

train_data1['origin'] = 'train'
submission['origin'] = 'test'
matrix = pd.concat([train_data1, submission], ignore_index=True, sort=False)
#print(matrix)

matrix.drop(['prob'], axis=1, inplace=True)
# 连接user_info表，通过user_id关联
matrix = matrix.merge(user_info, on='user_id', how='left')
# 使用merchant_id（原列名seller_id）
user_log.rename(columns={'seller_id':'merchant_id'}, inplace=True)
# 格式化
user_log['user_id'] = user_log['user_id'].astype('int32')
user_log['merchant_id'] = user_log['merchant_id'].astype('int32')
user_log['item_id'] = user_log['item_id'].astype('int32')
user_log['cat_id'] = user_log['cat_id'].astype('int32')
user_log['brand_id'].fillna(0, inplace=True)
user_log['brand_id'] = user_log['brand_id'].astype('int32')
user_log['time_stamp'] = pd.to_datetime(user_log['time_stamp'], format='%H%M')
# 1 for <18; 2 for [18,24]; 3 for [25,29]; 4 for [30,34]; 5 for [35,39]; 6 for [40,49]; 7 and 8 for >= 50; 0 and NULL for unknown
matrix['age_range'].fillna(0, inplace=True)
# 0:female, 1:male, 2:unknown
matrix['gender'].fillna(2, inplace=True)
matrix['age_range'] = matrix['age_range'].astype('int8')
matrix['gender'] = matrix['gender'].astype('int8')
matrix['label'] = matrix['label'].astype('str')

matrix['user_id'] = matrix['user_id'].astype('int32')
matrix['merchant_id'] = matrix['merchant_id'].astype('int32')
del user_info, train_data1
gc.collect()
print(matrix)

# User特征处理
groups = user_log.groupby(['user_id'])
# 用户交互行为数量 u1
temp = groups.size().reset_index().rename(columns={0:'u1'})
matrix = matrix.merge(temp, on='user_id', how='left')
# 使用agg 基于列的聚合操作，统计唯一值的个数 item_id, cat_id, merchant_id, brand_id
#temp = groups['item_id', 'cat_id', 'merchant_id', 'brand_id'].nunique().reset_index().rename(columns={'item_id':'u2', 'cat_id':'u3', 'merchant_id':'u4', 'brand_id':'u5'})
temp = groups['item_id'].agg([('u2', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
temp = groups['cat_id'].agg([('u3', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
temp = groups['merchant_id'].agg([('u4', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
temp = groups['brand_id'].agg([('u5', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')

# 时间间隔特征 u6 按照小时
temp = groups['time_stamp'].agg([('F_time', 'min'), ('L_time', 'max')]).reset_index()
temp['u6'] = (temp['L_time'] - temp['F_time']).dt.seconds/3600
matrix = matrix.merge(temp[['user_id', 'u6']], on='user_id', how='left')
# 统计操作类型为0，1，2，3的个数
temp = groups['action_type'].value_counts().unstack().reset_index().rename(columns={0:'u7', 1:'u8', 2:'u9', 3:'u10'})
matrix = matrix.merge(temp, on='user_id', how='left')
#print(matrix)

# 商家特征处理
groups = user_log.groupby(['merchant_id'])
# 商家被交互行为数量 m1
temp = groups.size().reset_index().rename(columns={0:'m1'})
matrix = matrix.merge(temp, on='merchant_id', how='left')
# 统计商家被交互的user_id, item_id, cat_id, brand_id 唯一值
temp = groups['user_id', 'item_id', 'cat_id', 'brand_id'].nunique().reset_index().rename(columns={'user_id':'m2', 'item_id':'m3', 'cat_id':'m4', 'brand_id':'m5'})
matrix = matrix.merge(temp, on='merchant_id', how='left')
# 统计商家被交互的action_type 唯一值
temp = groups['action_type'].value_counts().unstack().reset_index().rename(columns={0:'m6', 1:'m7', 2:'m8', 3:'m9'})
matrix = matrix.merge(temp, on='merchant_id', how='left')
# 按照merchant_id 统计随机负采样的个数
# temp = train_data[train_data['label']==-1].groupby(['merchant_id']).size().reset_index().rename(columns={0:'m10'})
# matrix = matrix.merge(temp, on='merchant_id', how='left')
#print(matrix)

# 按照user_id, merchant_id分组
groups = user_log.groupby(['user_id', 'merchant_id'])
temp = groups.size().reset_index().rename(columns={0:'um1'}) #统计行为个数
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')
temp = groups['item_id', 'cat_id', 'brand_id'].nunique().reset_index().rename(columns={'item_id':'um2', 'cat_id':'um3', 'brand_id':'um4'}) #统计item_id, cat_id, brand_id唯一个数
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')
temp = groups['action_type'].value_counts().unstack().reset_index().rename(columns={0:'um5', 1:'um6', 2:'um7', 3:'um8'})#统计不同action_type唯一个数
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')
temp = groups['time_stamp'].agg([('first', 'min'), ('last', 'max')]).reset_index()
temp['um9'] = (temp['last'] - temp['first']).dt.seconds/3600
temp.drop(['first', 'last'], axis=1, inplace=True)
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left') #统计时间间隔
#print(matrix)

#用户购买点击比
matrix['r1'] = matrix['u9']/matrix['u7'] 
#商家购买点击比
matrix['r2'] = matrix['m8']/matrix['m6'] 
#不同用户不同商家购买点击比
matrix['r3'] = matrix['um7']/matrix['um5']
matrix.fillna(0, inplace=True)
# # 修改age_range字段名称为 age_0, age_1, age_2... age_8
temp = pd.get_dummies(matrix['age_range'], prefix='age')
matrix = pd.concat([matrix, temp], axis=1)
temp = pd.get_dummies(matrix['gender'], prefix='g')
matrix = pd.concat([matrix, temp], axis=1)
matrix.drop(['age_range', 'gender'], axis=1, inplace=True)

print(matrix)

# 分割训练数据和测试数据
train_data = matrix[matrix['origin'] == 'train'].drop(['origin'], axis=1)
test_data = matrix[matrix['origin'] == 'test'].drop(['label', 'origin'], axis=1)
train_X, train_y = train_data.drop(['label'], axis=1), train_data['label']

print(train_y)
del temp, matrix
gc.collect()




In [0]:
# 使用机器学习工具
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import classification_report
import xgboost as xgb
# 将训练集进行切分，20%用于验证
X_train, X_valid, y_train, y_valid = train_test_split(train_X, train_y, test_size=.2)

# 使用XGBoost
model = xgb.XGBClassifier(
    max_depth=10,
    n_estimators=500,
    min_child_weight=300, 
    colsample_bytree=0.8, 
    subsample=0.8, 
    eta=0.3,    
    seed=42    
)
model.fit(
    X_train, y_train,
    eval_metric='auc', eval_set=[(X_train, y_train), (X_valid, y_valid)],
    verbose=True,
    #早停法，如果auc在10epoch没有进步就stop
    early_stopping_rounds=10 
)


model.fit(X_train, y_train)

prob = model.predict_proba(test_data)
submission['prob'] = pd.Series(prob[:,1])
submission.drop(['origin'], axis=1, inplace=True)

In [0]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import seaborn as sns
import graphviz

from imblearn.over_sampling import RandomOverSampler
from imblearn.pipeline import Pipeline

from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.model_selection import GridSearchCV, train_test_split

from sklearn.linear_model import LogisticRegression, Ridge, Lasso, ElasticNet
import lightgbm as lgb
import xgboost as xgb
#import catboost as cat

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, BaggingClassifier, ExtraTreesClassifier

from sklearn.metrics import roc_curve, auc, confusion_matrix, roc_auc_score
from sklearn.tree import export_graphviz
import warnings 
warnings.filterwarnings('ignore')

In [0]:
def plot_roc_curve(fprs, tprs):
    
    tprs_interp = []
    aucs = []
    mean_fpr = np.linspace(0, 1, 100)
    f, ax = plt.subplots(figsize=(8, 8))
    
    # Plotting ROC for each fold and computing AUC scores
    for i, (fpr, tpr) in enumerate(zip(fprs, tprs), 1):
        tprs_interp.append(np.interp(mean_fpr, fpr, tpr))
        tprs_interp[-1][0] = 0.0
        roc_auc = auc(fpr, tpr)
        aucs.append(roc_auc)
        ax.plot(fpr, tpr, lw=1, alpha=0.3, label='ROC Fold {} (AUC = {:.3f})'.format(i, roc_auc))
        
    # Plotting ROC for random guessing
    plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r', alpha=0.8, label='Random Guessing')
    
    mean_tpr = np.mean(tprs_interp, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    std_auc = np.std(aucs)
    
    # Plotting the mean ROC
    ax.plot(mean_fpr, mean_tpr, color='b', label='Mean ROC (AUC = {:.3f} $\pm$ {:.3f})'.format(mean_auc, std_auc), lw=2, alpha=0.8)
    
    # Plotting the standard deviation around the mean ROC Curve
    std_tpr = np.std(tprs_interp, axis=0)
    tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
    tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
    ax.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2, label='$\pm$ 1 std. dev.')
    
    ax.set_xlabel('False Positive Rate', size=15, labelpad=20)
    ax.set_ylabel('True Positive Rate', size=15, labelpad=20)
    ax.tick_params(axis='x', labelsize=15)
    ax.tick_params(axis='y', labelsize=15)
    ax.set_xlim([-0.05, 1.05])
    ax.set_ylim([-0.05, 1.05])

    ax.set_title('ROC Curves of Folds', size=20, y=1.02)
    ax.legend(loc='lower right', prop={'size': 13})
    
    plt.show()

In [0]:
def plot_feature_importances(feature_importances, title, feature_names):
    feature_importances = 100.0*(feature_importances/max(feature_importances))
    index_sorted = np.flipud(np.argsort(feature_importances))
    pos = np.arange(index_sorted.shape[0])+0.5
    
    fig, ax = plt.subplots(figsize=(16,4))
    plt.bar(pos,feature_importances[index_sorted])
    for tick in ax.get_xticklabels():
        tick.set_rotation(90)
    plt.xticks(pos,feature_names[index_sorted])
    plt.ylabel('Relative Importance')
    plt.title(title)
    plt.show() 

In [0]:
target = train_y
train = train_X
train, X_valid, target, y_valid = train_test_split(train_X, train_y, test_size=.5)
test = test_data
print(target)

In [0]:
def get_optimizer_params(model, model_params, train=train, test=test):
    gridsearch = GridSearchCV(model, model_params, scoring='roc_auc', cv=5)
    gridsearch.fit(train, target)
    best_score = gridsearch.best_score_
    print("Best score: %0.3f" % best_score)
    print("Best parameters set:")
    best_parameters = gridsearch.best_estimator_.get_params()
    for param_name in sorted(gridsearch.param_grid.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))
#     print(best_parameters)
    return best_parameters

def get_model_result(model, model_params, model_name, scaler=None, test_size=0.5, train=train, test=test, feature_importance=False, gridsearch=True):
    if scaler is not None:
        train = scaler.fit_transform(train)
        test = scaler.fit_transform(test)

    if gridsearch is True:
        best_params = get_optimizer_params(model, model_params)
        model.set_params(**best_params)
    X_train, X_val, y_train, y_val = train_test_split(train, target, test_size=test_size, random_state=2020, stratify=target)
    model.fit(X_train, y_train)
    prob_y_val = model.predict_proba(X_val)[:,1] if hasattr(model, 'predict_proba') else model.predict(X_val)
    trn_fpr, trn_tpr, trn_thresholds = roc_curve(y_val, prob_y_val)
    best_score = auc(trn_fpr, trn_tpr)
    best_parameters = model.get_params()
    plot_roc_curve([list(trn_fpr)], [list(trn_tpr)])
        
        
    if feature_importance is True:
        plot_feature_importances(model.feature_importances_, 'Importance of Features', train.columns)
    result = pd.DataFrame()
    result['user_id'] = user_id
    result['Attrition'] = pd.DataFrame(model.predict_proba(test)[:,1] if hasattr(model, 'predict_proba') else model.predict(test))
    result[['user_id', 'Attrition']].to_csv(f'result-{model_name}.csv', index=False, float_format='%.8f')
    print('result of predict:\n', result.head())
    return best_score, best_parameters

In [0]:
data = pd.concat([train, test]).corr() ** 2
data = np.tril(data, k=-1)
data[data==0] = np.nan
print(data)

In [0]:
figure, ax = plt.subplots(figsize=(10, 10))
sns.heatmap(np.sqrt(data), annot=False, cmap='viridis', ax=ax)

In [0]:
data = train.corrwith(target).agg('square')

figure, ax = plt.subplots(figsize=(10, 10))
data.agg('sqrt').plot.bar(ax=ax)
del data

In [0]:
model = Ridge()
model_params = {'alpha': [0.01, 0.05, 0.1, 0.5, 1, 2, 5, 7], 'tol': [0.0001, 0.001, 0.01, 0.1]}
model_params = {'alpha': [2], 'tol': [0.0001]}
start = time.time()
best_score_ridge, best_params_ridge = get_model_result(model, model_params, 'ridge', StandardScaler())
print(time.time()-start)
print('best_score_ridge,', best_score_ridge)
print('best_params_ridge,', best_params_ridge)
del model

In [0]:
model = Lasso()
model_params = {'alpha': np.logspace(-10, -6, 50)}
model_params = {'alpha': [1e-10]}
print(model_params)
start = time.time()
best_score_lasso, best_params_lasso = get_model_result(model, model_params, 'lasso', StandardScaler())
print(time.time() - start)
del model

In [0]:
model = ElasticNet()
model_params = {'alpha': np.logspace(-10, -4, 10), 'l1_ratio': np.logspace(-10, -4, 10)}
#model_params = {'alpha': [0.0001], 'l1_ratio': [1e-10]}
start = time.time()
best_score_elasticnet, best_params_elasticnet = get_model_result(model, model_params, 'elasticnet', StandardScaler())
print(time.time() - start)
del model 

In [0]:
model = LogisticRegression(class_weight='balanced')
model_params = {
    'penalty': ['l1', 'l2'], 
    'C': [0.4, 0.5, 0.6],
    'solver':['liblinear', 'lbfgs', 'newton-cg', 'sag', 'saga']
    }
# model_params = {
#     'penalty': ['l1'], 
#     'C': [0.5],
#     'solver':['liblinear']
# }
start = time.time()
best_score_lgr,best_params_lgr = get_model_result(model, model_params,'logisticregression', StandardScaler()) 
print(time.time() - start)
del model 

In [0]:
model = SVC()
model_params = {
    'C':[0.1,1,5,10,50,100, 200],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'gamma':[1,0.1,0.01,0.001]
}
best_score_svc, best_params_svc = get_model_result(model, model_params, 'SVC', StandardScaler())
del model 

In [0]:
model = xgb.XGBClassifier(random_state=2020,tree_method='gpu_hist', silent=1, booster='gbtree', objective='binary:logistic')
model_params = {
    'booster':['gbtree'],
    'colsample_bytree': [0.5, 0.8],
    'subsample': [0,3, 0.5],
    'learning_rate': [0.075, 0.01],
    'objective': ['binary:logistic'],
    'max_depth': [ 7, 8, 9],
    'num_parallel_tree': [0.1, 1, 10],
    'min_child_weight': [0.2, 0.8],
}
# model_params = {
#     'colsample_bytree': [0.5],
#     'subsample': [0.5],
#     'learning_rate': [0.075],
#     'max_depth': [9],
#     'num_parallel_tree': [1],
#     'min_child_weight': [0.2],
# }

start = time.time()
best_score_xgboost, best_params_xgboost = get_model_result(model, model_params, 'xgboost')
print(time.time()-start)
del model  

In [0]:
model = DecisionTreeClassifier(random_state=2020)
model_params = {
    'max_features': [0.8, 1.0], 
    'max_depth': [8, 9, 10],
    'min_samples_leaf':[30, 40, 50]
} 
# model_params = {
#     'max_features': [1.0], 
#     'max_depth': [9],
#     'min_samples_leaf':[30]
# }
start = time.time()
best_score_dtc, best_params_dtc = get_model_result(model, model_params, 'dtc', None, feature_importance=True)
print(time.time() - start)
del model 

In [0]:
model = RandomForestClassifier(random_state=2020)
model_params = {
    'n_estimators':[500],
    'n_jobs':[-1],
    'max_features': [0.5, 0.6], 
    'max_depth': [8, 9, 10],
    'min_samples_leaf':[8, 10],
#     'random_state':[2020]
}
model_params = {
    'n_estimators':[200],
    'max_features': [0.6], 
    'max_depth': [10],
    'min_samples_leaf':[10],
}
start = time.time()
best_score_rfc, best_params_rfc = get_model_result(model, model_params, 'rfc', feature_importance=True)
print(time.time() - start)
del model 

In [0]:
model = lgb.LGBMClassifier(random_state=2020, device='gpu', gpu_platform_id=0, gpu_device_id=0, silent=1)
model_params = {
    'n_estimators': [200, 300, 400],
    'learning_rate': [0.01, 0.1, 0.5],
    'num_leaves':[10,100,400],
    'colsample_bytree':[0.5,0.8, 1.0],
    'subsample':[0.3,0.5,0.9],
    'max_depth':[7, 10, 15],
    'reg_alpha':[0.01, 0.2, 0.5],
    'reg_lambda':[0.01, 0.3, 0.8],
    'min_split_gain':[0.01, 0.1],
    'min_child_weight': [1,2,4],
}
start = time.time()
best_score_lightgbm, best_params_lightgbm = get_model_result(model, model_params, 'lightgbm')
print(time.time()-start)
del model 